In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")

In [2]:
X, y = make_classification(n_samples=5000, n_features=5, n_redundant=2,
                           n_classes=2, weights=[0.7], class_sep=0.7, random_state=15)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=15)

X_test, X_cv, y_test, y_cv = train_test_split(X_test, y_test, test_size=0.5, random_state=15)


In [3]:
def decision_function(X_cv, support_vectors, dual_coef, intercept, gamma):
    dec_fun = []
    
    for xq in X_cv:                

        dual_form = 0
        for i in range(len(support_vectors)):
            dual_form += dual_coef[i] * np.exp(-gamma * (np.linalg.norm(support_vectors[i] - xq) ** 2))

        dual_form = dual_form + intercept

        if dual_form > 0:
            dec_fun.append(1)
        else:
            dec_fun.append(-1)

    return dec_fun

In [4]:
gamma = 0.001

clf = SVC(C= 100,gamma= gamma)
clf.fit(X_train, y_train)

f_cv = decision_function(X_cv, clf.support_vectors_, clf.dual_coef_[0], clf.intercept_, gamma)
print('----------Custom Decision Function----------')
print(f_cv)

N_positive = len(list(filter(lambda a: (a < 0), f_cv)))
N_negative = len(list(filter(lambda a: (a > 0), f_cv)))

print('----------Custom Decision Function Positive and Negative Count----------')
print(N_positive)
print(N_negative)


print('----------Classifier\'s Decision Function----------')
clf_f_cv = clf.decision_function(X_cv)
print(clf_f_cv)

print('----------Classifier\'s Decision Function Positive and Negative Count----------')
clf_N_positive = len(list(filter(lambda a: (a <= 0), clf_f_cv)))
clf_N_negative = len(list(filter(lambda a: (a > 0), clf_f_cv)))

print(clf_N_positive)
print(clf_N_negative)

----------Custom Decision Function----------
[-1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, 1, 1, 

<font color="green"><b>We can see that the results are same from both Custom and SVC's decision function<b></font>

In [5]:
y_positive = N_positive + 1 /N_positive + 2
y_negative = 1 / N_negative + 2

print(y_positive)
print(y_negative)

704.0014245014245
2.0033557046979866


In [6]:
class EpochLoss:
  def __init__(self, epoch, loss, weight, intercept):
    self.epoch = epoch
    self.loss = loss
    self.weight = weight
    self.intercept = intercept
    


class SGDLogisticRegression:           
    
    
    def log_loss(self, w, b, X, Y):
        N = len(X)
        sum_log = 0
        for i in range(N):

            y = 0
            if Y[i] == 1:
                y = y_positive
            else:
                y = y_negative

            sum_log += y * np.log10(self.sigmoid(X[i], w, b)) + (1 - y) * np.log10(1 - self.sigmoid(X[i], w, b))
        
        return -1 * sum_log/N
        
    
    def sigmoid(self, x, w, b):
        
        z = np.dot(x, w.T) + b
        sig = 1 / (1 + np.exp(-z))
        
        return sig
        
        
    
    def fit(self, X, Y, eta0, alpha, num_iteration):
        
        N = len(X)        
        lst_train = []
        
        
        #Initial w and b
        w = np.zeros_like(X[0])
        b = 0

        for epoch in range(1, num_iteration):                                
                                        
            
            error = self.log_loss(w, b, X, Y)

            #Updating weights and intercept
            w = (1 - (alpha * eta0)/N) * w + alpha * error
            b = (b + alpha * error)           
            
            
            #Checking if previous epoch for train data is having the same value then breaking the epoch loop
            round_upto = 3
            if len(lst_train) > 0:

              found = False
              for loss in lst_train:
                if loss.loss == round(error, round_upto):
                  found = True
                  break
              
              if found == False:
                lst_train.append(EpochLoss(epoch, round(error, round_upto), w, b))                
              else:
                break

            else:
                lst_train.append(EpochLoss(epoch, round(error, round_upto), w, b))                
            
            

            print('Epoch= %d, Bias= %.3f, Avg. Loss= %.3f' % (epoch, b, error))
        
        self.coef = w 
        self.intercept = b
        self.lstEpochLoss_train = lst_train        

        return self
    
    def pred(self, X, w, b):
      
      N = len(X)
      predict = []
      for i in range(N):
        if self.sigmoid(X[i], np.array(w), b) >= 0.5:
          predict.append(1)
        else:
          predict.append(0)
          
      return np.array(predict)

In [7]:
eta0  = 0.001 #lambda
alpha = 0.001 #learning rate
num_iterations = 50

sgd_logistic_reg = SGDLogisticRegression()
model = sgd_logistic_reg.fit(f_cv, y_cv, eta0, alpha, num_iterations)

print('\n--------Weight--------')
print(model.coef)
print('\n--------Bias--------')
print(model.intercept)

Epoch= 1, Bias= 0.000, Avg. Loss= 0.301
Epoch= 2, Bias= 0.001, Avg. Loss= 0.251
Epoch= 3, Bias= 0.001, Avg. Loss= 0.209
Epoch= 4, Bias= 0.001, Avg. Loss= 0.174
Epoch= 5, Bias= 0.001, Avg. Loss= 0.145
Epoch= 6, Bias= 0.001, Avg. Loss= 0.121
Epoch= 7, Bias= 0.001, Avg. Loss= 0.101
Epoch= 8, Bias= 0.001, Avg. Loss= 0.084
Epoch= 9, Bias= 0.001, Avg. Loss= 0.070
Epoch= 10, Bias= 0.002, Avg. Loss= 0.059
Epoch= 11, Bias= 0.002, Avg. Loss= 0.049
Epoch= 12, Bias= 0.002, Avg. Loss= 0.041
Epoch= 13, Bias= 0.002, Avg. Loss= 0.034
Epoch= 14, Bias= 0.002, Avg. Loss= 0.028
Epoch= 15, Bias= 0.002, Avg. Loss= 0.024
Epoch= 16, Bias= 0.002, Avg. Loss= 0.020
Epoch= 17, Bias= 0.002, Avg. Loss= 0.016
Epoch= 18, Bias= 0.002, Avg. Loss= 0.014
Epoch= 19, Bias= 0.002, Avg. Loss= 0.011
Epoch= 20, Bias= 0.002, Avg. Loss= 0.010
Epoch= 21, Bias= 0.002, Avg. Loss= 0.008
Epoch= 22, Bias= 0.002, Avg. Loss= 0.007
Epoch= 23, Bias= 0.002, Avg. Loss= 0.006
Epoch= 24, Bias= 0.002, Avg. Loss= 0.005
Epoch= 25, Bias= 0.002, A

In [8]:
f_test = decision_function(X_test, clf.support_vectors_, clf.dual_coef_[0], clf.intercept_, gamma)

In [9]:
y_pred = sgd_logistic_reg.pred(f_test, model.coef, model.intercept)
print(y_pred)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 